## **Dataset**

In this assignment, we will do data munging and analysis on a dataset about board games. The dataset is from Kaggle, at https://www.kaggle.com/andrewmvd/board-games.

In [ ]:
import pandas as pd

games = pd.read_csv('bgg_dataset.csv', delimiter=';', decimal=",")

We set `display.max_rows` to `None` for now so that we can see more values in code output. 

In [ ]:
pd.set_option('display.max_rows', None)

Let's gain some basic understanding of the dataset by using `info()`, `head()`, and `describe()`.

In [ ]:
games.info()

In [ ]:
games.head()

In [ ]:
games.describe()

## **Need for data cleaning and preprocessing**

The results of these several functions indicate a few needs for cleaning and preprocessing the data:

1) The columns `ID`, `Year Published` and `Owned Users` should be integers, but they are floating point numbers.

2) There are null values in various columns. 

3) It seems there could be wrong values. For instance, the minimal value in `Max Players` is 0. What kind of game is that if it allows at most zero player? 

4) The values in columns `Mechanics` and `Domains` are comma-separated lists. We need to parse these values and get the individual items from the lists.


Let's find out which columns have null values. This could be derived from the `Non-Null Count` in the output of `games.info()`. But there are much simpler ways.

## **Task 1: For each column, find the number of rows with null value in that column.**  

In [ ]:
# Code for Task 1
games.isnull().sum()

Since column `ID` has null values, it couldn't be used for uniquely identifying games. Hence, let's take it out. 

## **Task 2: Remove the `ID` column from the `DataFrame` `games`.**

In [ ]:
# Code for Task 2
games = games.drop(columns="ID")
games

## **Task 3: Replace null values in column `Year Published` by 5000. Replace null values in column `Owned Users` by -1.** 

The column `Year Published` has negative values, as `games.descrie()` shows. It actually has 0 in its values too. Hence, we are using a year in the future (5000) to indicate the dataset doesn't provide the value for a game. The column `Owned Users` has 0s too. We thus use -1 to indicate missing values.

In [ ]:
# Code for Task 3 : code block for replaceing null values in column Year Published by 5000.
games["Year Published"].fillna("5000", inplace=True)

In [ ]:
# Code for Task 3: code block for replacing null values in column Owned Users by -1.
games["Owned Users"].fillna("-1", inplace=True)

## **Task 4: Convert the data type of column ``Year Pubblished`` to integer. Convert the data type of column ``Owned Users`` to integer too.**

In [ ]:
# Code for Task 4: code block for converting the data type of column ``Year Pubblished`` to integer.
games['Year Published'] = games['Year Published'].astype(int)

In [ ]:
# Code for Task 4: code block for converting the data type of column ``Owned Users`` to integer. 
games['Owned Users'] = games['Owned Users'].astype(int)

After we finish Tasks 1 to 4, run `games.info()`, `games.head()`, and `games.describe()` again, to verify we have achieved the goals. 

In [ ]:
games.info()

In [ ]:
games.head()

In [ ]:
games.describe()

Earlier we noticed the existence of value 0 in certain columns which shouldn't have such values. Let's find out how prevalent the prolem is. 

## **Task 5: For each column, show how many rows have 0 as the value.**



In [ ]:
# Code for Task 5
games.isin([0]).sum()


## **Task 6: Get the number of games published in each year. Sort the years by frequency, in descending order.** 

In [ ]:
# Code for Task 6
games.groupby('Year Published').size().sort_values(ascending=False)

The oldest game was from 3500 BC. Let's find out which game it is. 

## **Task 7: Find the name of the oldest game, based on column `Year Published`.**

In [ ]:
# Code for Task 7
games.nsmallest(1,['Year Published']).iloc[0]['Name']

In the output of Task 6, we see that in general the number of games published in a year has been steadily increasing. However, Year 0 appears to be an outlier, as it has 185 games according to the dataset. This doesn't seem right. To further verify, let's do the following Task 8.

## **Task 8: Find how many games in total have been publisehd before 1900. Do not include Year 0 in the count.**

If code is correct, it shall return 111. We can thus conclude the statistics for Year 0 cannot be correct. Most likely this is because 0 was used to indicate unknown/missing publishing year when the dataset was created. How confusing that is. This reminds us it is important to make good choices in dealing with missing values. 

In [ ]:
# Code for Task 8
games[(games['Year Published']<1900) & (games['Year Published']!=0)].count()['Year Published']

For the same reason, we believe value 0 in all other columns are not reliable either. Our next task will replace 5000 in `Year Published` and -1 in `Owned Users` by 0. Remember they were actually null values (Task 3). Later we will ignore them together with all 0 values in our analysis.

## **Task 9: Replace 5000 in `Year Published` and -1 in `Owned Users` by 0.**

In [ ]:
# Code for Task 9
games['Year Published'] = games['Year Published'].replace(5000, 0)
games['Owned Users'] = games['Owned Users'].replace(-1, 0)

Also, let's take a look at the frequency of each value in column `Min Players`. If we write a piece of code to find out, the code will be similiar to the one in Task 6. Hence, we don't provide it here. Instead, we directly provide the values, as follows. 

In [ ]:
pd.Series({0: 46, 1: 3270, 2: 14076, 3: 2365, 4: 474, 5: 57, 6: 21, 7: 14, 8: 17, 9: 1, 10: 2}, name = 'Min Players')

Our next task attempts to examine the relationship between `Min Players` and popularity of games measured by ownership. 

## **Task 10: For each value of `Min Players`, find the average `Owned Users` for games with the corresponding `Min Players` value. Exclude the games with values 0, 9, 10 in `Min Players` and value 0 in `Owned Users`.**

As we discussed in Task 8, we don't trust the value 0 in any of the columns. We can thus ignore the games with 0 in `Min Players`. Furthermore, there are only 1 and 2 games for `Min Players` 9 and 10, respectively. The statistics of these games won't be meaningful. When we focus on the rest of the games in the output of Task 10, we will observe a general pattern of decresing ownership by minimum required players. That is probably not surprising, since it is easier to find people to play games with less required players.  

In [ ]:
# Code for Task 10
games[((games["Min Players"] != 0) & (games["Min Players"] < 9)) & (games["Owned Users"] !=0)].groupby("Min Players")["Owned Users"].mean()

The pattern from Task 10 appears to have some exceptions, in games with `Min Players` being 5 and 8. Let's take a further look. Before we continue, let's change `display.max_rows` to 50.

In [ ]:
pd.set_option('display.max_rows', 50)

## **Task 11: Find the top-5 owned games for each group of games based on `Min Players`. Ideally, we should also exclude the games with values 0, 9, 10 in `Min Players`, like what we did in Task 10. It is fine if we don't do it here.**

From the output, we see that there are some quite popular games with 5 and 8 `Min Players`.

In [ ]:
# Code for Task 11 
games[((games["Min Players"] != 0) & (games["Min Players"] < 9))].sort_values(by=["Owned Users"], ascending = False).groupby('Min Players').head(5)

## **Task 12: Produce a pivot table using `Rating Average` as the rows (i.e., index) and `Complexity Average` as the columns. The cells of the pivot table should show the number of games having the corresponding rating average and complexity average. Since both `Rating Average` and `Complexity Average` are floating point numbers, we should use bins on both. Let's create ten bins [1,2], (2,3], (3,4], (4,5], (5,6], (6,7], (7,8], (8,9], (9,10] on `Rating Average` and four bins [1,2], (2,3], (3,4], (4,5] on `Complexity Average`.**

The output of `games.describe()` shows that column `Complexity Average` has value 0. Based on our earlier analysis, we shouldn't put much faith in this value. Besides 0, the smallest value in that column is 1. Therefore we decide to have four bins for `Complexity Average`. 

In [ ]:
# Code for Task 12
rating = pd.cut(games['Rating Average'], bins=range(1,11), include_lowest=True, right=True)
complexity = pd.cut(games['Complexity Average'], bins=range(1,6), include_lowest=True, right=True)

games.pivot_table('Name', rating, complexity, aggfunc='count') 

The pivot table suggests a positive correlation between these two columns. As the complexity of games increases, the rating also tends to increase. Perhaps this is intuitive. For complex games to have a market, it needs to be of higher quality. 

In fact, we can directly calculate the correlation using `corr`, as follows. The value of 0.5 in Pearson correlation coefficient suggests a fairly large positive correlation.

In [ ]:
g = games[games['Complexity Average']>0]

g['Rating Average'].corr(g['Complexity Average'], method='pearson')

In the next task, we are going to produce a similar pivot table, focusing on `Owned Users`.  

## **Task 13: Produce another pivot table using `Rating Average` and `Complexity Average`, with the same binning. However, this time the cells of the pivot table should show average `Owned Users` of games matching the corresponding rating average and complexity average.**

In [ ]:
# Code for Task 13
games.pivot_table('Owned Users', rating, complexity)

This pivot table also shows some interesting patterns. At every rating tier till (7, 8], simpler games enjoy larger ownerships. However, for the really good games with ratings greater than 8, players are not afraid of their complexity. In fact, the more complex games in this rating tier get owned by more players.

Now we will process the `Mechanics` and `Domains` columns. They store values as strings. Each string is a comma-separated list of items. The following code will turn `Mechanics` into a DataFrame itself, with each column corresponding to a unique item from the comma-separated lists. Similarly, we are creating a new DataFrame for the `Domains` column.

In [ ]:
mechanics = games['Mechanics'].str.get_dummies(sep=", ")
domains = games['Domains'].str.get_dummies(sep=", ")

mechanics = pd.concat([games['Name'], mechanics], 1)
domains = pd.concat([games['Name'], domains], 1)

Let's take a look at the columns in the new DataFrame `mechanics`. The values are 1 and 0, i.e., essentially Boolean, indicating whether a game uses the corresponding mechanics or not. This is also called *one-hot encoding*. 


In [ ]:
mechanics.head()

Similarly the new DataFrame `domains` uses one-hot encoding to record the games' domain types. 

In [ ]:
domains.head()

Let's find out which are the most common game domains and which are the least common ones.

## **Task 14: For each domain, list its frequency, i.e., the number of games belonging to that domain.**

In [ ]:
# Code for Task 14
(domains == 1).sum(axis=0)[1:]

## **Task 15: Find out the average `Complexity Average` of games that belong to `Wargames`. We should exclude games with 0 on `Complexity Average`.**

In [ ]:
# Code for Task 15
pd.merge(games[games['Complexity Average']>0], domains[domains['Wargames']==1], left_index=True, right_index=True)['Complexity Average'].mean()

If we perform the same task on `Children's Games`, we will get 1.175. These two are the two types of games with the largest and smallest average `Complexity Average`. If we want to calculate this for every type of games, the following code does it.

In [ ]:
results = \
    (games[games['Complexity Average']>0].set_index(games.columns.drop('Domains',1).tolist())
    .Domains.str.split(', ', expand=True)
    .stack()
    .reset_index()
    .rename(columns={0:'domain'})
    .loc[:,['domain','Owned Users', 'Rating Average', 'Complexity Average']]
    .groupby('domain').agg({'Owned Users':['mean'], 'Rating Average':['mean'], 'Complexity Average':['mean']})
    )
results 